In [13]:
import json
import pandas as pd

In [14]:
with open('C:/CODE/SIC_BigData_Real-Estate-Price-Prediction/data/ChungCu/ChungCu_raw-dataset.json','r',encoding='utf-8') as file:
    data = json.load(file)
df1 = pd.json_normalize(data)
df1.drop_duplicates(inplace=True)

In [15]:
df1.drop(columns=['DiaChi','City','District','PhapLy','HuongNha','NoiThat'])

,DienTich,MucGia,SoPhongNgu,SoToilet,Lat,Long,HuongBanCong,MatTien,DuongVao,SoTang
0,50 m²,45 triệu/m²,1 phòng,1 phòng,10.739294,106.644394,NaN,NaN,NaN,NaN
1,50 m²,"3,7 tỷ",1 phòng,1 phòng,10.756248,106.691849,Đông - Nam,NaN,NaN,NaN
2,73 m²,Thỏa thuận,2 phòng,2 phòng,21.015852,105.809235,Nam,NaN,NaN,NaN
3,"33,24 m²",1 tỷ,1 phòng,NaN,21.026758,105.841194,NaN,NaN,NaN,NaN
4,110 m²,50 triệu/m²,3 phòng,2 phòng,20.987026,105.786835,Đông - Bắc,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
38616,68 m²,"3,35 tỷ",2 phòng,2 phòng,10.749306,106.635384,NaN,NaN,NaN,NaN
38617,53 m²,"1,3 tỷ",2 phòng,2 phòng,20.951185,107.035522,NaN,NaN,NaN,NaN
38619,46 m²,910 triệu,2 phòng,1 phòng,10.899458,106.832153,NaN,NaN,NaN,NaN
38620,51 m²,Thỏa thuận,1 phòng,1 phòng,20.859999,106.684479,Nam,NaN,NaN,NaN


In [16]:
#xử lý diện tích
df1["DienTich"] = df1["DienTich"].str.replace(' m²','').str.replace(',', '.').str.strip()

In [17]:
#Hàm tính toán giá Đất
def clean_price(row):
    price = row['MucGia']
    area = row['DienTich']
    
    # Đổi kiểu cột DienTich thành float
    area = float(area)
    # Đảm bảo price là chuỗi
    if isinstance(price, (float, int)):
        return price

    price = str(price)
    
    #chuyển đổi đơn vị giá -> đồng
    if 'triệu/m²' in price:
        price_value = float(price.replace(' triệu/m²', '').replace(',', '.').strip()) * 1000000
        return price_value * area
    elif 'thỏa thuận' in price:
        return price
    elif 'nghìn/m²' in price:
        price_value = float(price.replace(' nghìn/m²', '').replace(',', '.').strip()) * 1000
        return price_value * area
    elif 'tỷ/m²' in price:
        price_value = float(price.replace(' tỷ/m²', '').replace(',', '.').strip()) * 1000000000
        return price_value * area
    elif 'triệu' in price:
        price_value = float(price.replace(' triệu', '').replace(',', '.').strip()) * 1000000
        return price_value
    elif 'tỷ' in price:
        price_value = float(price.replace(' tỷ', '').replace(',', '.').strip()) * 1000000000
        return price_value
    elif 'nghìn' in price:
        price_value = float(price.replace(' nghìn', '').replace(',', '.').strip()) * 1000
        return price_value

    return price

# Áp dụng hàm vào DataFrame
df1['MucGia'] = df1.apply(clean_price, axis=1)
print(df1['MucGia'])

0        2250000000.0
1        3700000000.0
2          Thỏa thuận
3        1000000000.0
4        5500000000.0
             ...     
38616    3350000000.0
38617    1300000000.0
38619     910000000.0
38620      Thỏa thuận
38636    2550000000.0
Name: MucGia, Length: 22827, dtype: object


In [18]:
#Gán giá trị trung vị cho 'thỏa thuận'
# Tính giá trị trung vị
median_price = df1[df1['MucGia'] != 'Thỏa thuận']['MucGia'].astype(float).median()

# Thay thế giá trị "thỏa thuận"
df1['MucGia'] = df1['MucGia'].replace('Thỏa thuận', median_price)

C:\Users\quanp\AppData\Local\Temp\ipykernel_11328\1240423143.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df1['MucGia'] = df1['MucGia'].replace('Thỏa thuận', median_price)


In [19]:
#xử lý diện tích mặt tiền
df1["MatTien"] = df1["MatTien"].str.replace(' m','').str.replace(',','.').str.strip()

#xử lý diện tích đường vào
df1["DuongVao"] = df1["DuongVao"].str.replace(' m','').str.replace(',','.').str.strip()

#xử lý diện tích số phòng ngủ
df1["SoPhongNgu"] = df1["SoPhongNgu"].str.replace(' phòng','').str.strip()

#xử lý số phòng Toilet
df1["SoToilet"] = df1["SoToilet"].str.replace(' phòng','').str.strip()

#xử lý số tầng
df1["SoTang"] = df1["SoTang"].str.replace(' tầng','').str.strip()